# JIRA hours in WFD
1. define a date range and Jira project
2. pull hours from Tempo for that date range/project
3. Add only columns needed to dictionary (key,seconds,assignee,issuetype)

4. add jira ticket columns (customer, parentId, parentSummary, epicId, epicSummary)
5. join Tempo and Jira dataframes

## Todo
* Jira needs to return all issuetypes (or figure out some other grouping scheme)
* authentication shouldn't be visible

# 1. define date range and Jira project

In [25]:
fromDate = '2017-01-01'
toDate = '2017-04-30'
projectKey = 'WFD'
projectId = '12900'

#https://levelsbeyond.atlassian.net/rest/tempo-timesheets/3/worklogs?
#dateFrom=2017-04-01&dateTo=2017-04-30&projectId=12900&projectKey=WFD
tempo_url = 'https://levelsbeyond.atlassian.net/rest/tempo-timesheets/3/worklogs?' + \
    'dateFrom={0}&dateTo={1}&projectId={2}&projectKey={3}'

#maxRecords=1000
#there are about 2300 in WFD, will need to page through them all making 3 calls
#https://levelsbeyond.atlassian.net/rest/api/2/search?
#jql=project=WFD%20AND%20issuetype%20in%20(Story,UAT,Epic)&expand=names
jira_url = 'https://levelsbeyond.atlassian.net/rest/api/2/search?' + \
'jql=project={0}%20AND%20issuetype%20in%20(Story,UAT,Epic,Sub-Task)&expand=names' \
+ '&fields=key,summary,customfield_13500,issuetype,parent&maxResults=1000'

#single is too slow
jira_single_url = 'https://levelsbeyond.atlassian.net/rest/api/2/issue/{0}'

# 2. pull hours from Tempo for the date range/project

In [26]:
import requests
import json

url = tempo_url.format(fromDate,toDate,projectId,projectKey)
#print(url)
r = requests.get(url, auth=('rjohnson', 'Miter9le'))
#print(r.status_code)
json_return = json.loads(r.text)
#print(json_return)

# 3. Clean up the Tempo data

In [27]:
jira_dict = {}
for entry in json_return:    
    jira_dict[entry['issue']['key']] = {'key': entry['issue']['key'], 
                                        'issueType': entry['issue']['issueType']['name'],
                                        'timeSpentSeconds': entry['timeSpentSeconds'],
                                        'author': entry['author']['name']}   
#print(jira_dict)

# 4. add jira ticket columns (customer, parentId, parentSummary, epicId, epicSummary)

In [48]:
#print(jira_dict['WFD-698']['issueType'])

import requests
import json

#for k,v in jira_dict.items():
#    print(k,v)
    
#    url = jira_single_url.format(k)
#    print(url)

#    r = requests.get(url, auth=('rjohnson', 'Miter9le'))
#    print(r.status_code)

#    json_return = json.loads(r.text)

#    print(json_return['key'])





print(jira_dict[json_return['key']])

print(json_return['fields']['issuetype']['name'])
try: 
    print(json_return['fields']['parent']['key'])
    print(json_return['fields']['parent']['fixVersion']['name'])
    print(json_return['fields']['parent']['summary'])
    print(json_return['fields']['parent']['status']['name'])
except:
   pass 

print(json_return['fields']['customfield_13500'])
print(json_return['fields']['customfield_10008'])

print(json_return)

#json_return = json_return['issues']
#print(json_return)

WFD-698 {'key': 'WFD-698', 'issueType': 'Testing Task', 'timeSpentSeconds': 1800, 'author': 'mmclaughlin'}
200
WFD-698
WFD-311 {'key': 'WFD-311', 'issueType': 'Time Tracking Task', 'timeSpentSeconds': 3600, 'author': 'ddewhurst'}
200
WFD-311
WFD-711 {'key': 'WFD-711', 'issueType': 'Testing Task', 'timeSpentSeconds': 8100, 'author': 'mmclaughlin'}
200
WFD-711
WFD-708 {'key': 'WFD-708', 'issueType': 'Testing Task', 'timeSpentSeconds': 2700, 'author': 'mmclaughlin'}
200
WFD-708
WFD-707 {'key': 'WFD-707', 'issueType': 'Testing Task', 'timeSpentSeconds': 1800, 'author': 'mmclaughlin'}
200
WFD-707
WFD-327 {'key': 'WFD-327', 'issueType': 'Time Tracking Task', 'timeSpentSeconds': 3600, 'author': 'tseale'}
200
WFD-327
WFD-394 {'key': 'WFD-394', 'issueType': 'Story', 'timeSpentSeconds': 1800, 'author': 'mmclaughlin'}
200
WFD-394
WFD-312 {'key': 'WFD-312', 'issueType': 'Time Tracking Task', 'timeSpentSeconds': 1200, 'author': 'mmclaughlin'}
200
WFD-312
WFD-667 {'key': 'WFD-667', 'issueType': 'Tes

KeyboardInterrupt: 

# 4. move Jira data to a dataframe

In [26]:
from pandas.io.json import json_normalize
import pandas as pd
from IPython.display import display

temp_df = json_normalize(json_return)
jira_df = temp_df.set_index(['key'])


jira_df['parentKey'] = jira_df['fields.parent.key']


#jira_df = pd.DataFrame(json_return)
del jira_df['id']
del jira_df['self']
del jira_df['fields.customfield_13500']
del jira_df['fields.customfield_13500.id']
del jira_df['fields.customfield_13500.self']
del jira_df['expand']
del jira_df['fields.issuetype.iconUrl']
del jira_df['fields.issuetype.id']
del jira_df['fields.issuetype.self']
del jira_df['fields.issuetype.avatarId']
del jira_df['fields.issuetype.subtask']
#del jira_df['parent']
#del jira_df['']

jira_df.sort_index(inplace=True)
display(jira_df)

,fields.customfield_13500.value,fields.issuetype.description,fields.issuetype.name,fields.parent.fields.issuetype.avatarId,fields.parent.fields.issuetype.description,fields.parent.fields.issuetype.iconUrl,fields.parent.fields.issuetype.id,fields.parent.fields.issuetype.name,fields.parent.fields.issuetype.self,fields.parent.fields.issuetype.subtask,...,fields.parent.fields.status.statusCategory.id,fields.parent.fields.status.statusCategory.key,fields.parent.fields.status.statusCategory.name,fields.parent.fields.status.statusCategory.self,fields.parent.fields.summary,fields.parent.id,fields.parent.key,fields.parent.self,fields.summary,parentKey
key,,,,,,,,,,,,,,,,,,,,,
WFD-1403,NaN,The sub-task of the issue,Sub-Task,11715.0,A user story. Created by JIRA Software - do no...,https://levelsbeyond.atlassian.net/secure/view...,7,Story,https://levelsbeyond.atlassian.net/rest/api/2/...,False,...,3.0,done,Done,https://levelsbeyond.atlassian.net/rest/api/2/...,SCOPED (.5): Create MP4 Proxies,60666,WFD-1136,https://levelsbeyond.atlassian.net/rest/api/2/...,(.5) Create parent for create proxy,WFD-1136
WFD-1404,NaN,The sub-task of the issue,Sub-Task,11715.0,A user story. Created by JIRA Software - do no...,https://levelsbeyond.atlassian.net/secure/view...,7,Story,https://levelsbeyond.atlassian.net/rest/api/2/...,False,...,3.0,done,Done,https://levelsbeyond.atlassian.net/rest/api/2/...,SCOPED(2): Rewrite Collection Restore For Howa...,62626,WFD-1393,https://levelsbeyond.atlassian.net/rest/api/2/...,SA Preview,WFD-1393
WFD-1406,NaN,The sub-task of the issue,Sub-Task,11715.0,A user story. Created by JIRA Software - do no...,https://levelsbeyond.atlassian.net/secure/view...,7,Story,https://levelsbeyond.atlassian.net/rest/api/2/...,False,...,3.0,done,Done,https://levelsbeyond.atlassian.net/rest/api/2/...,SCOPED(2): Rewrite Collection Restore For Howa...,62626,WFD-1393,https://levelsbeyond.atlassian.net/rest/api/2/...,SA Postview,WFD-1393
WFD-1407,NaN,The sub-task of the issue,Sub-Task,11715.0,A user story. Created by JIRA Software - do no...,https://levelsbeyond.atlassian.net/secure/view...,7,Story,https://levelsbeyond.atlassian.net/rest/api/2/...,False,...,3.0,done,Done,https://levelsbeyond.atlassian.net/rest/api/2/...,SCOPED (.5): Create MP4 Proxies,60666,WFD-1136,https://levelsbeyond.atlassian.net/rest/api/2/...,(.5) Failure workflow,WFD-1136
WFD-1408,NaN,The sub-task of the issue,Sub-Task,11715.0,A user story. Created by JIRA Software - do no...,https://levelsbeyond.atlassian.net/secure/view...,7,Story,https://levelsbeyond.atlassian.net/rest/api/2/...,False,...,3.0,done,Done,https://levelsbeyond.atlassian.net/rest/api/2/...,SCOPED(2): Rewrite Collection Restore For Howa...,62626,WFD-1393,https://levelsbeyond.atlassian.net/rest/api/2/...,Look over Black Pearl Documentation for Bulk E...,WFD-1393
WFD-1409,NaN,The sub-task of the issue,Sub-Task,11715.0,A user story. Created by JIRA Software - do no...,https://levelsbeyond.atlassian.net/secure/view...,7,Story,https://levelsbeyond.atlassian.net/rest/api/2/...,False,...,3.0,done,Done,https://levelsbeyond.atlassian.net/rest/api/2/...,SCOPED(2): Rewrite Collection Restore For Howa...,62626,WFD-1393,https://levelsbeyond.atlassian.net/rest/api/2/...,Gathering the Needed Documentation From Black ...,WFD-1393
WFD-1410,NaN,The sub-task of the issue,Sub-Task,11715.0,A user story. Created by JIRA Software - do no...,https://levelsbeyond.atlassian.net/secure/view...,7,Story,https://levelsbeyond.atlassian.net/rest/api/2/...,False,...,3.0,done,Done,https://levelsbeyond.atlassian.net/rest/api/2/...,SCOPED(2): Rewrite Collection Restore For Howa...,62626,WFD-1393,https://levelsbeyond.atlassian.net/rest/api/2/...,Prepare Assets for Bulk Call,WFD-1393
WFD-1411,NaN,The sub-task of the issue,Sub-Task,11715.0,A user story. Created by JIRA Software - do no...,https://levelsbeyond.atlassian.net/secure/view...,7,Story,https://levelsbeyond.atlassian.net/rest/api/2/...,False,...,3.0,done,Done,https://levelsbeyond.atlassi

# 5. join Tempo and Jira dataframes

In [6]:
full_df = pd.concat([tempo_df, jira_df], axis=1, join_axes=[tempo_df.index])
display(full_df)

,author.displayName,comment,dateStarted,issue.issueType.name,issue.summary,timeSpentSeconds,fields.customfield_13500.value,fields.issuetype.description,fields.issuetype.name,fields.summary
key,,,,,,,,,,
WFD-1000,Tyler Seale,Testing with Jason,2017-02-06T13:56:00.000,Sub-Task,(.5) Testing,1800,NaN,NaN,NaN,NaN
WFD-1001,Tyler Seale,"Currently do not have a VM installed, usually ...",2017-02-03T18:08:00.000,Sub-Task,unexpected,1800,NaN,NaN,NaN,NaN
WFD-1002,Dylan Dewhurst,Helped Jason find out why the Archive workflow...,2017-02-07T21:32:00.000,Sub-Task,Unexpected,1800,NaN,NaN,NaN,NaN
WFD-1020,Dylan Dewhurst,Made changes for removing second archive optio...,2017-02-09T14:48:00.000,Sub-Task,Make changes,3120,NaN,NaN,NaN,NaN
WFD-1021,Dylan Dewhurst,Finished testing and uploaded to Github.,2017-02-09T16:36:00.000,Sub-Task,Testing,2820,NaN,NaN,NaN,NaN
WFD-1022,Mitchell McLaughlin,Modifying workflows,2017-02-09T00:00:00.000,Sub-Task,Modify Workflows,1800,NaN,NaN,NaN,NaN
WFD-1023,Mitchell McLaughlin,testing changes,2017-02-09T00:00:00.000,Sub-Task,Test Changes,1800,NaN,NaN,NaN,NaN
WFD-1024,Mitchell McLaughlin,Moving renaming functionality so that all work...,2017-02-09T00:00:00.000,Sub-Task,Moving the renaming steps into the subflow so ...,1800,NaN,NaN,NaN,NaN
WFD-1025,Tyler Seale,created cron parent wf,2017-02-09T17:29:00.000,Sub-Task,(.25) create cron to find assets to run on,900,NaN,NaN,NaN,NaN


In [6]:
from pandas.io.json import json_normalize
from IPython.display import display

temp_df = json_normalize(json_return)
tempo_df = temp_df.set_index(['issue.key'])
tempo_df.index.names = ['key']

#remove unnecessary columns
del tempo_df['author.avatar']
del tempo_df['author.name']
del tempo_df['author.self']
del tempo_df['billedSeconds']
del tempo_df['id']
del tempo_df['issue.id']
del tempo_df['issue.issueType.iconUrl']
del tempo_df['issue.projectId']
del tempo_df['issue.remainingEstimateSeconds']
del tempo_df['issue.self']
del tempo_df['self']
del tempo_df['workAttributeValues']
del tempo_df['worklogAttributes']

tempo_df.sort_index(inplace=True)
display(tempo_df)

,author.displayName,comment,dateStarted,issue.issueType.name,issue.summary,timeSpentSeconds
key,,,,,,
WFD-1000,Tyler Seale,Testing with Jason,2017-02-06T13:56:00.000,Sub-Task,(.5) Testing,1800
WFD-1001,Tyler Seale,"Currently do not have a VM installed, usually ...",2017-02-03T18:08:00.000,Sub-Task,unexpected,1800
WFD-1002,Dylan Dewhurst,Helped Jason find out why the Archive workflow...,2017-02-07T21:32:00.000,Sub-Task,Unexpected,1800
WFD-1020,Dylan Dewhurst,Made changes for removing second archive optio...,2017-02-09T14:48:00.000,Sub-Task,Make changes,3120
WFD-1021,Dylan Dewhurst,Finished testing and uploaded to Github.,2017-02-09T16:36:00.000,Sub-Task,Testing,2820
WFD-1022,Mitchell McLaughlin,Modifying workflows,2017-02-09T00:00:00.000,Sub-Task,Modify Workflows,1800
WFD-1023,Mitchell McLaughlin,testing changes,2017-02-09T00:00:00.000,Sub-Task,Test Changes,1800
WFD-1024,Mitchell McLaughlin,Moving renaming functionality so that all work...,2017-02-09T00:00:00.000,Sub-Task,Moving the renaming steps into the subflow so ...,1800
WFD-1025,Tyler Seale,created cron parent wf,2017-02-09T17:29:00.000,Sub-Task,(.25) create cron to find assets to run on,900
